### Define Query Data Step

In [1]:
%%writefile query_data.py

import gen3
import os

guids = ['dg.MD1R/ea669b5e-ae51-40ba-b375-ed23a9cd1855',
         'dg.MD1R/a745ed98-0cb9-4537-826b-13b2e354e8bb',
         'dg.MD1R/e604979a-c71b-4ec6-b8a0-959837b86384',
         'dg.MD1R/b5cee98d-46ff-4438-aa00-90727a383340',
         'dg.MD1R/8a5a5579-7925-432d-a614-3ed208f1c182',
         'dg.MD1R/33034812-47f3-4c0e-b60b-fa7a2a04ecda',
         'dg.MD1R/5ca987c5-c660-4785-a67d-a3424cc8ec6e',
         'dg.MD1R/44148117-1858-49ef-b30f-d239abfaff80',
         'dg.MD1R/9ea205e8-a774-4318-a323-95eadda9bc5c',
         'dg.MD1R/09ece36f-a0fa-48e8-8fc2-62110eaae570']

for guid in guids:
    os.system(f"gen3 drs-pull object {guid} --output-dir ../../../midrc_files/midrc_dicom")



Overwriting query_data.py


### Define DICOM to PNG Step

Still work in process - need to adjust how the function steps through the directory path of all dicom files

In [2]:
%%writefile convert_dcm_to_png.py


import pydicom
import numpy as np
from PIL import Image
import os


base_folder =  '../../../midrc_files/midrc_dicom/COVID-19-NY-SBU'

at_end = False

for file in os.listdir(base_folder):
    if file.endswith(".dcm"):
        image_path = os.path.join(base_folder, file)

        ds = pydicom.dcmread(image_path)

        new_image = ds.pixel_array.astype(float)

        scaled_image = np.uint8((np.maximum(new_image, 0) / new_image.max()) * 255.0)

        final_image = Image.fromarray(scaled_image)

        out_path = os.path.join('../../../midrc_files/midrc_png/', file[:-3], 'png')
        final_image.save(out_path)


Overwriting convert_dcm_to_png.py


### Defin CICOM to JPEG Step 

Still work in process - need to adjust how the function steps through the directory path of all dicom files

In [3]:
%%writefile convert_dcm_to_jpeg.py


import pydicom
import numpy as np
from PIL import Image

image_path = 'COVID-19-NY-SBU/A117394/10-08-1900-CT ABD AND PELVIS WITH IV CONT-39755/9.000000-CTA 0.5 CE-40834/1-0163.dcm'

ds = pydicom.dcmread(image_path)
    
new_image = ds.pixel_array.astype(float)

scaled_image = np.uint8((np.maximum(new_image, 0) / new_image.max()) * 255.0)

final_image = Image.fromarray(scaled_image)

final_image.save(image_path.rsplit('/', 1)[1][:-3] + 'jpg')   

Overwriting convert_dcm_to_jpeg.py


### Define Extract Metadata Step

In [4]:
%%writefile extract_image_metadata.py

import pandas as pd
import os
from dicom_csv import join_tree

base_folder =  '../../../midrc_files/midrc_dicom/COVID-19-NY-SBU'
metadata_df = pd.DataFrame()

file_folders = os.listdir(path = base_folder)

for folder in file_folders:
        path = base_folder + '/' + folder

        meta = join_tree(path, verbose=2)
        metadata_df = pd.concat([metadata_df, meta])

metadata_df.to_csv('../../../midrc_files/extracted_metadata.csv')



Overwriting extract_image_metadata.py


### Define Workflow

Still work in progress - need to make workflow perform query step before all other processes

In [9]:
%%writefile main.nf
#!/usr/bin/env nextflow

process QueryData {
    
    tag 'initial_task'
    

    script:
    """
    python3 ${baseDir}/query_data.py 
    """
}

process ConvertToJPEG {

   
    // publishDir "${baseDir}/converted_images/jpeg", mode: 'copy'
    
    tag 'convert_to_jpeg'

    script:
    """
    python3 ${baseDir}/convert_dcm_to_jpeg.py 
    """
}

process ConvertToPNG {
    
    
    // publishDir "${baseDir}/converted_images/png", mode: 'copy'
    
    tag 'convert_to_png'


    script:
    """
    python3 ${baseDir}/convert_dcm_to_png.py
    """
}

process ExtractMetadata {
    
    
    // publishDir "${baseDir}/extracted_metadata", mode: 'copy'
    
    tag 'extract_metadata'
    

    script:
    """
    python3 ${baseDir}/extract_image_metadata.py 
    """
}


// Define the entry workflow (initial workflow for Nextflow to run)
workflow {

    QueryData()
    
    // ConvertToJPEG()
    
    // ConvertToPNG()
    
    // ExtractMetadata()
    
}

Overwriting main.nf


### Define Environment

Still work in progress - need to properly set up environment

In [10]:
%%writefile environment.yml
name: MIDRC_CT_Scan_Workflow_env
channels:
  - defaults
dependencies:
  - python
  - pip
  - pip:
    - gen3
    - numpy
    - pandas
    - pydicom
    - pillow
    - dicom-csv
    - os

Overwriting environment.yml


### Define Workflow Config

In [11]:
%%writefile nextflow.config

// enables the processes to be run in a conda environment
// conda.enabled = true

process {
    conda {
        envCreate = true
        envName = 'MIDRC_CT_Scan_Workflow_env'
        envFile = 'environment.yml'
    }
}

params {
    midrc_guid = "dg.MD1R/ea669b5e-ae51-40ba-b375-ed23a9cd1855"    
    outdir = "/results"
}

Overwriting nextflow.config


### Run Workflow

In [13]:
!nextflow run main.nf

N E X T F L O W  ~  version 22.10.6
Launching `main.nf` [nasty_chandrasekhar] DSL2 - revision: 23fc6d37e0
[-        ] process > ExtractMetadata [  0%] 0 of 1

executor >  local (1)
[7b/81d3da] process > ExtractMetadata (extract_me... [  0%] 0 of 1

executor >  local (1)
[7b/81d3da] process > ExtractMetadata (extract_me... [100%] 1 of 1 ✔

